# Fuzzy Search 

## PyData DC 2016

Jiaqi Liu

https://github.com/jiaqi216/pydata_dc


    

**whoami**

- Jiaqi Liu
- Data Scientist @ Capital One
- NYC based
- work mostly at the intersection of data science and engineering
- https://github.com/jiaqi216/pydata_dc


**Fuzzy Match Algorithms**

- "Distance between words"
- Soundex 
- Levenshtein
- n-gram
- NLTK/Word2Vec

**Soundex**

- phonetic algorithm
- index by sound as pronounced in English
- assigns a soundex coding 
- ideal for spelling inconsistencies

**American Soundex Coding**

[http://www.archives.gov/research/census/soundex.html](http://www.archives.gov/research/census/soundex.html)

every soundex code is a letter and three numbers

| Number | Letter |
|---------|----------|
|1|B,F,P,V|
|2|C,G,J,K,Q,S,X,Z|
|3|D,T|
|4|L|
|5|M,N|
|6|R|

Ignore A,E,I,O,U,H,W,Y

In [46]:
import jellyfish as j

a=j.soundex('WASHINGTON')
print(a)
b=j.soundex('WUSHINGTON')
print(b)

W252
W252


In [8]:
a=j.soundex('LGA')
print(a)
b=j.soundex('LAGUARDIA')
print(b)

L200
L263


In [48]:
a=j.soundex('WORD#123')
print(a)
a2=j.soundex('WORD')
print(a2)

W630
W630


In [49]:
#Homonyms
a=j.soundex('accept')
print(a)
a2=j.soundex('except')
print(a2)

A213
E213


In [50]:
a=j.soundex('forth')
print(a)
a2=j.soundex('fourth')
print(a2)

F630
F630


Soundex with PostgreSQL

https://www.postgresql.org/docs/9.1/static/fuzzystrmatch.html

`CREATE EXTENSION fuzzystrmatch;`

In [53]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://hhu373@localhost/cities')
connection = engine.connect()

In [92]:
from sqlalchemy.sql import text

query = "select soundex('Anne'), soundex('Ann'), difference('Anne','Ann')"
res = engine.execute(text(query))
res.fetchall()

[('A500', 'A500', 4)]

"The difference function converts two strings to their Soundex codes and then reports the number of matching code positions. Since Soundex codes have four characters, the result ranges from zero to four, with zero being no match and four being an exact match." - PostgreSQL documentation

In [121]:
query = """
        SELECT    alt_spelling 
        FROM      fuzzy_names 
        WHERE     soundex(alt_spelling) = soundex('WASHINGTON DC')"""

In [122]:
res = engine.execute(text(query))
res.fetchall()

[(' WSHINGTONDC',), (' Washington D.C.',), (' WSHINGTON DC',)]

In [123]:
query = """
        SELECT    alt_spelling 
        FROM      fuzzy_names 
        WHERE     difference(alt_spelling, 'WASHINGTON DC') > 2"""

In [124]:
res = engine.execute(text(query))
res.fetchall()

[('Tuscon, AZ',),
 (' Washinton DC',),
 ('Los Angelos,CA',),
 ('Las Angeles, CA',),
 ('Las Angeles, CA',),
 (' WSHINGTONDC',),
 (' Wushinton D.C.',),
 (' Washington D.C.',),
 (' WSHINGTON DC',)]

In [125]:
#https://www.kaggle.com/kaggle/hillary-clinton-emails
#7946 emails
query = """ SELECT     metadatasubject
            FROM       emails 
            WHERE      difference(metadatasubject, 'qatar')> 2"""
res = engine.execute(text(query))
res.fetchall()

[('QDDR',),
 ('QDDR',),
 ('QDDR',),
 ('2010-2011 QDDR TERMS OF REFERENCE',),
 ('2010-2011 QDDR TERMS OF REFERENCE',),
 ('WATER',),
 ('WATER',),
 ('WATER',),
 ('WATER',),
 ('QATAR',),
 (' LETTER',),
 (' LETTER',),
 (' LETTER',),
 (' LETTER',),
 ('QDDR',),
 ('LETTER',),
 ('QDDR',),
 ("QDDR MARKS BOLD STEP FORWARD IN STRENGTHENING AMERICA'S CIVILIAN POWER",),
 ('LETTER',),
 ('WATER',),
 ('WATER',),
 ('QDDR',),
 ('WATER',),
 ('QDDR FOLLOWUP',),
 ('WATER',),
 ('QDDR/THANKS',),
 (' LETTER',),
 ('4 QDDR DRAFT CHAPTERS',),
 ('QDDR',),
 ('QDDR',),
 ('QDDR',),
 ('QDDR',),
 ('QUOTER ABOUT MITCH MCCONNELL',),
 ('QDDR BREAKFAST',),
 ('QDDR COVER LETTER FROM S',),
 ('LETTER',)]

**Soundex**
- Soundex is pretty easy to implement
- Computationally fast
- only works on ASCII characters (no foreign languages)
- How do you calculate distance

**Levenshtein distance**
- also call edit distance
- accounts for how many characters you have to change to have the same string
- computationally fast (can handle real time processing)
- pairwise comparison

In [98]:
import Levenshtein as l

l.distance('SMYTHE', 'SMITH')

2

In [99]:
l.distance('café', 'cafe')

1

Pitfall: Comparing Addresses

In [100]:
str99 = '99 Broadway'
str100 = '100 Broadway'
str999 = '999 Broadway'

l.distance(str99, str100)

3

- weighing numbers differently from letters

Longer Strings

In [101]:
str1='Hello world, python is great'
str2='Hello world, python is good'
l.distance(str1,str2)

4

In [102]:
import Levenshtein as l
str1='great'
str2='good'
l.distance(str1,str2)

4

- counting raw edits penalizes long strings: use a ratio of edits to length


**Damerau-Levenshtein**
- like Levenshtein but accounts for transposition of adjacent characters

In [103]:
j.damerau_levenshtein_distance('recieve', 'receive')

1

In [104]:
j.levenshtein_distance('recieve', 'receive')

2

In [105]:
import csv
def load_cities_list():
    cities = []
    with open('data/misspelled_cities.csv') as data_file:
        reader = csv.reader(data_file, delimiter='|')
        for correct, wrong in reader:
            cities.append({'correct': correct,'wrong': wrong})
    return cities

In [106]:
for city in load_cities_list():
    print(j.levenshtein_distance(city['correct'], city['wrong']))

9
1
1
2
2
1
2
2
1
2
1
4
11
1
2
12
2
1
5
1
1
2
2
1
1
1
1
1
1
1
2
1
1
1
1
1
1
3
10


**n-gram/Trigram**

- groupings of letters (takes into more context)
- proper unit of analysis (1-gram, 2-gram, 3-gram)
- slower to implement - need to calculate n-gram for each string

Sentences

In [107]:
def ngram(tokens, n):
    grams =[tokens[i:i+n] for i in range(len(tokens)-(n-1))]
    return grams

In [108]:
sentence_gram = "The quick brown fox jumped over a lazy dog".split()
grams = ngram(sentence_gram, 3)

for gram in grams:
    print(gram)


['The', 'quick', 'brown']
['quick', 'brown', 'fox']
['brown', 'fox', 'jumped']
['fox', 'jumped', 'over']
['jumped', 'over', 'a']
['over', 'a', 'lazy']
['a', 'lazy', 'dog']


Words

In [109]:
def ngram(tokens, n):
    grams =[tokens[i:i+n] for i in range(len(tokens)-(n-1))]
    return grams

In [110]:
word_gram = "pydatadc"
grams = ngram(word_gram, 3)

for gram in grams:
    print(gram)


pyd
yda
dat
ata
tad
adc


Scoring Similarity: Jaccard similarity

In [111]:
def get_sim(a_tri,b_tri):
    intersect = len(set(a_tri) & set(b_tri))
    union = len(set(a_tri) | set(b_tri))
    return float(intersect)/(union)

In [112]:
print(grams)
get_sim(grams, grams)

['pyd', 'yda', 'dat', 'ata', 'tad', 'adc']


1.0

Word2Vec: uses cosine distance
cosine distance between two vectors is nice but like have to find a way to quantify

In [114]:
a_gram = ngram('receved data', 3)
b_gram = ngram('received date', 3)
get_sim(a_gram, b_gram)

0.5

Trigam Search with Postgres

In [1]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://hhu373@localhost/cities')
connection = engine.connect()

In [17]:
#https://www.postgresql.org/docs/9.1/static/pgtrgm.html
#create extension pg_trgm;
query="""
    SELECT 
                a.alt_spelling, 
                similarity(lower(a.alt_spelling), :city) as similarity 
    FROM        fuzzy_names as a 
    WHERE       lower(a.alt_spelling) % :city
    ORDER BY    similarity DESC"""

In [18]:
from sqlalchemy.sql import text

city_name = 'washington'
res = engine.execute(text(query), city=city_name)

In [19]:
res.fetchall()

[(' Washington D.C.', 0.733333),
 (' Washinton DC', 0.5),
 (' WSHINGTON DC', 0.5),
 (' WSHINGTONDC', 0.4375)]

In [87]:
query = """ select metadatasubject,
similarity(lower(metadatasubject), :subject) as similarity
from emails where lower(metadatasubject) % :subject order by similarity desc
"""
res = engine.execute(text(query), subject='secretary')
res.fetchall()

[("SECRETARY'S EMAIL", 0.588235),
 ('SECRETARY SEBELIUS', 0.588235),
 ("SECRETARY'S EMAIL", 0.588235),
 ("SECRETARY'S EMAIL", 0.588235),
 ('FOR THE SECRETARY', 0.555556),
 ('FOR THE SECRETARY', 0.555556),
 ("SECRETARY'S REMARKS", 0.526316),
 ('NOTE FOR SECRETARY', 0.526316),
 ('SECRETARY OF AWESOME', 0.47619),
 ('SECRETARY IN TRIPOLI', 0.47619),
 ("THE SECRETARY'S SPEECH", 0.47619),
 ('SECRETARY OF AWESOME', 0.47619),
 ('SECRETARY OF AWESOME', 0.47619),
 ('NOTE FOR THE SECRETARY', 0.434783),
 ("SECRETARY'S VIDEO SPEECH", 0.434783),
 ("SECRETARY'S VIDEO SPEECH", 0.434783),
 ('MESSAGE FOR THE SECRETARY', 0.384615),
 ('PLEASE PASS TO THE SECRETARY', 0.37037),
 ('CALL WITH SECRETARY CLINTON', 0.37037),
 ('PLEASE PASS TO THE SECRETARY', 0.37037),
 ('CALL WITH SECRETARY CLINTON', 0.37037),
 ('NOTE FOR SECRETARY CLINTON', 0.37037),
 ('CALL WITH SECRETARY CLINTON', 0.37037),
 ('THANK YOU SECRETARY CLINTON', 0.357143),
 ('LETTER TO SECRETARY CLINTON', 0.357143),
 ('PLEASE CONVEY TO THE SECRETAR

Some things to note:
   - the data set i'm working with is not large
   - gist and gin indexes for trigrams
   
   ` CREATE INDEX trgm_idx ON table_name USING gist (t gist_trgm_ops);`
   
   or 
   
   ` CREATE INDEX trgm_idx ON table_name USING gin (t gin_trgm_ops);`
    

**Other Similarity Metrics**
- NLTK: wordnet
- Word2Vec: uses cosine distance
    - cosine distance between two vectors


In [20]:
from nltk.corpus import wordnet

In [21]:
word1 = wordnet.synsets("blue")
word2 = wordnet.synsets("green")
word1[0].wup_similarity(word2[0])

0.875

In [ ]:
#sample data set from: http://mattmahoney.net/dc/text8.zip
import word2vec
word2vec.word2phrase('text/text8', 'text/text8-phrases', verbose=True)
word2vec.word2vec('text/text8-phrases', 'text/text8.bin', size=100, verbose=True)

In [126]:
import word2vec
model = word2vec.load('text/text8.bin')
model['coffee']

array([-0.10188702,  0.22015855, -0.0104067 , -0.11283471,  0.17996578,
        0.03414147,  0.00380119,  0.02513149,  0.05538284,  0.03358232,
        0.0046195 , -0.00982001,  0.14096248, -0.03092421,  0.02837873,
       -0.10945819,  0.06230062, -0.09781564,  0.05339954, -0.0469771 ,
       -0.24697879, -0.01727498, -0.07510021,  0.05243083,  0.13334657,
       -0.0842028 , -0.15239957,  0.04965223,  0.15619534,  0.07072332,
        0.04465058,  0.04456624, -0.13465466,  0.07669131, -0.04658106,
        0.03763594,  0.08352708, -0.06075072,  0.03647474, -0.03983858,
       -0.05725538,  0.19554052, -0.0493384 , -0.0864112 , -0.01019673,
        0.11497202, -0.04002611, -0.01038931,  0.09136914, -0.06539407,
        0.06577577,  0.02044055,  0.05330006, -0.08380378,  0.08494037,
       -0.03088019, -0.15074271, -0.00925445, -0.05506656, -0.06991567,
       -0.18828158, -0.15286787,  0.0320463 ,  0.03634158,  0.00368489,
       -0.05393973, -0.06775434, -0.24145566, -0.04210955,  0.06

In [115]:
def get_similar_words(word):
    indexes, metrics = model.cosine(word)
    return model.generate_response(indexes, metrics).tolist()

In [119]:
get_similar_words('coffee')

[('cotton', 0.8697562574405371),
 ('wheat', 0.8625143596285044),
 ('beef', 0.8595074656304864),
 ('tea', 0.8569193660043646),
 ('bananas', 0.8564837525303834),
 ('poultry', 0.84936196362873),
 ('sugar', 0.8485541657571788),
 ('cocoa', 0.8464163883133844),
 ('rice', 0.8437539975509329),
 ('maize', 0.8435050129873531)]

In [120]:
get_similar_words('emails')

[('password', 0.7342604837846847),
 ('mail_server', 0.7157839344016247),
 ('irc_server', 0.71493902396743),
 ('email', 0.7148194115667482),
 ('cellphone', 0.7046435452801667),
 ('web_server', 0.7044565758834691),
 ('dhcp', 0.7036272137098812),
 ('x_chat', 0.7013826973226731),
 ('mysql', 0.7012250028951611),
 ('cross_platform', 0.6879122291851275)]

# Questions?
https://github.com/jiaqi216/pydata_dc